In [57]:
import tracemalloc
import time
import scanpy as sc
import pandas as pd
import scipy

import cupy as cp
import cupyx
import numpy as np
from tqdm import tqdm

import sys

from icecream import ic

from SEACells.core_copy import SEACells

In [58]:
num_cells = 1000

n_SEACells = num_cells // 75
build_kernel_on = "X_pca"  # key in ad.obsm to use for computing metacells

n_waypoint_eigs = (10)

In [59]:
ad = sc.read("/home/aparna/DATA/aparnakumar/150000_cells/mouse_marioni_150k.h5ad")
ad = ad[:num_cells]

In [60]:
model2 = SEACells(
    ad,
    use_gpu=False,
    use_sparse=True,
    build_kernel_on=build_kernel_on,
    n_SEACells=n_SEACells,
    n_waypoint_eigs=n_waypoint_eigs,
    convergence_epsilon=1e-5,
)

SPARSE AND NOT GPU
TRYING SEACellsCPU
Welcome to SEACells!


In [61]:
model4 = SEACells( 
    ad,
    use_gpu=True,
    use_sparse=True,
    build_kernel_on=build_kernel_on,
    n_SEACells=n_SEACells,
    n_waypoint_eigs=n_waypoint_eigs,
    convergence_epsilon=1e-5,
)

SPARSE AND GPU
TRYING SEACellsGPU
Welcome to SEACells GPU!


In [62]:
model2.construct_kernel_matrix()

Computing kNN graph using scanpy NN ...
Computing radius for adaptive bandwidth kernel...



Making graph symmetric...
Parameter graph_construction = union being used to build KNN graph...
Computing RBF kernel...



Building similarity LIL matrix...



Constructing CSR matrix...


In [63]:
K2 = model2.K

In [64]:
kernel_matrix = model2.kernel_matrix
cupyx_kernel_matrix = cupyx.scipy.sparse.csr_matrix(kernel_matrix)

kernel_matrix_T = kernel_matrix.T 
cupyx_kernel_matrix_T = cupyx_kernel_matrix.T


In [65]:
# Check if kernel_matrix and cupyx_kernel_matrix are equal 
print(np.allclose(kernel_matrix.todense(), cupyx_kernel_matrix.get().todense()))

True


In [66]:
# Check if kernel_matrix_T and cupyx_kernel_matrix_T are equal 
print(np.allclose(kernel_matrix_T.todense(), cupyx_kernel_matrix_T.get().todense()))

True


In [67]:
K = kernel_matrix @ kernel_matrix_T 
K_cupy = cupyx_kernel_matrix @ cupyx_kernel_matrix_T

In [68]:
# Check if K and K_cupy are equal 
print(np.allclose(K.todense(), K_cupy.get().todense()))

True


In [69]:
diff = (K_cupy.get().todense() - K.todense()).flatten().tolist()[0]

# Get the frequency of the elements in diff  
freq = {}
for item in diff:
    if (item in freq):
        freq[item] += 1
    else:
        freq[item] = 1

print(freq)

{8.881784197001252e-16: 225, 0.0: 979791, 1.1102230246251565e-16: 2528, -4.440892098500626e-16: 1993, -5.551115123125783e-17: 1971, -2.220446049250313e-16: 2997, 5.551115123125783e-17: 2078, 4.440892098500626e-16: 1862, -1.1102230246251565e-16: 2511, 2.220446049250313e-16: 2953, -2.7755575615628914e-17: 340, -8.881784197001252e-16: 262, 2.7755575615628914e-17: 340, 6.661338147750939e-16: 15, 1.3877787807814457e-17: 25, -1.3322676295501878e-15: 13, -6.661338147750939e-16: 27, 6.938893903907228e-18: 11, -1.3877787807814457e-17: 14, -6.938893903907228e-18: 6, -3.469446951953614e-18: 3, 1.7763568394002505e-15: 7, 1.3322676295501878e-15: 19, -3.3306690738754696e-16: 1, 3.3306690738754696e-16: 1, -1.7763568394002505e-15: 7}


In [70]:
model4.add_precomputed_kernel_matrix(cupyx_kernel_matrix)

In [71]:
K4 = model4.K

In [72]:
# Check if K and K4 are equal 
print(np.allclose(K.todense(), K4.get().todense()))

True


In [73]:
model2.initialize()

Building kernel on X_pca
Computing diffusion components from X_pca for waypoint initialization ... 
Determing nearest neighbor graph...


100%|██████████| 14/14 [00:00<00:00, 618.90it/s]

Done.
Sampling waypoints ...
Done.
Selecting 9 cells from waypoint initialization.
Initializing residual matrix using greedy column selection
Initializing f and g...
Selecting 4 cells from greedy initialization.
Randomly initialized A matrix.


Setting convergence threshold at 0.00056


In [74]:
A0 = model2.A0
B0 = model2.B0

archetypes = model2.archetypes 

In [75]:
k = len(archetypes)
n = K4.shape[0]
cols = cp.arange(k)
rows = cp.array(archetypes)
shape = (n, k)

In [76]:
B0_4 = cupyx.scipy.sparse.csr_matrix((cp.ones(len(rows)), (rows, cols)), shape=shape)

In [77]:
# Compare B0 and B0_4 
print(np.allclose(B0.todense(), B0_4.get().todense()))

True


In [78]:
A0_4 = cupyx.scipy.sparse.csr_matrix(A0)
B0_4 = cupyx.scipy.sparse.csr_matrix(B0) 

# Compare A0 and A0_4
print(np.allclose(A0.todense(), A0_4.get().todense()))

# Compare B0 and B0_4
print(np.allclose(B0.todense(), B0_4.get().todense()))


# Compute the sparsity of A0 and A0_4 
print("Sparsity of A0: ", A0.nnz / (A0.shape[0] * A0.shape[1]))
print("Sparsity of A0_4: ", A0_4.nnz / (A0_4.shape[0] * A0_4.shape[1]))

True
True
Sparsity of A0:  0.2129230769230769
Sparsity of A0_4:  0.2129230769230769


In [79]:
new_A_4 = model4._updateA(B0_4, A0_4)

print(new_A_4)

  (0, 0)	0.05411764705882353
  (0, 1)	0.048627450980392145
  (0, 2)	0.09098039215686275
  (0, 3)	0.09019607843137255
  (0, 4)	0.6964705882352941
  (0, 5)	0.09725490196078432
  (0, 6)	0.04784313725490198
  (0, 7)	0.11215686274509803
  (0, 8)	0.0823529411764706
  (0, 9)	0.09254901960784315
  (0, 10)	0.11215686274509803
  (0, 11)	0.12627450980392158
  (0, 12)	0.09019607843137255
  (0, 13)	0.07294117647058827
  (0, 14)	0.09803921568627454
  (0, 15)	0.08941176470588233
  (0, 16)	0.04235294117647058
  (0, 17)	0.11215686274509803
  (0, 18)	0.011764705882352943
  (0, 19)	0.11215686274509803
  (0, 20)	0.07843137254901959
  (0, 21)	0.10039215686274511
  (0, 22)	0.11529411764705882
  (0, 23)	0.12392156862745098
  (0, 24)	0.07372549019607844
  :	:
  (12, 975)	0.0603921568627451
  (12, 976)	0.08392156862745098
  (12, 977)	0.08470588235294117
  (12, 978)	0.0203921568627451
  (12, 979)	0.08392156862745098
  (12, 980)	0.05882352941176472
  (12, 981)	0.030588235294117642
  (12, 982)	0.01882352941176470

In [80]:
new_A_2 = model2._updateA(B0, A0)

print(new_A_2)

  (0, 0)	0.053333333333333344
  (1, 0)	0.022745098039215685
  (2, 0)	0.3513725490196078
  (3, 0)	0.06509803921568628
  (6, 0)	0.01568627450980392
  (7, 0)	0.018039215686274503
  (8, 0)	0.034509803921568626
  (9, 0)	0.3333333333333333
  (10, 0)	0.04235294117647059
  (11, 0)	0.049411764705882356
  (12, 0)	0.014117647058823528
  (0, 1)	0.048627450980392145
  (1, 1)	0.054901960784313725
  (2, 1)	0.03058823529411765
  (3, 1)	0.06745098039215686
  (4, 1)	0.0219607843137255
  (5, 1)	0.05176470588235294
  (6, 1)	0.014901960784313736
  (7, 1)	0.018823529411764704
  (8, 1)	0.035294117647058816
  (10, 1)	0.5654901960784314
  (11, 1)	0.04313725490196079
  (12, 1)	0.04705882352941177
  (0, 2)	0.09254901960784315
  (1, 2)	0.10117647058823531
  :	:
  (12, 997)	0.15450980392156863
  (0, 998)	0.07607843137254901
  (1, 998)	0.04705882352941178
  (2, 998)	0.0611764705882353
  (3, 998)	0.05411764705882353
  (4, 998)	0.05490196078431373
  (5, 998)	0.04156862745098039
  (6, 998)	0.04392156862745099
  (7, 99

In [81]:
# compare new_A_2 and new_A_4 

# print(np.allclose(new_A_2.todense(), new_A_4.get().todense()))

In [82]:
# diff = (new_A_2.todense() - new_A_4.get().todense()).flatten().tolist()[0] 

# # Get the frequency of the elements in diff
# freq = {}
# for item in diff:
#     if (item in freq):
#         freq[item] += 1
#     else:
#         freq[item] = 1

# print(freq)

In [83]:
# Set model2.A_ to new_A_2 
model2.A_ = new_A_2

# Set model4.A_ to new_A_4 
model4.A_ = new_A_4

# Compare model2.A_ and model4.A_ 
# print(np.allclose(model2.A_.todense(), model4.A_.get().todense()))

# Check the shapes of model2.A_ and model4.A_
print(model2.A_.shape)
print(model4.A_.shape)


(13, 1000)
(13, 1000)


In [84]:
# Set model2.B_ to B0 
model2.B_ = B0

# Set model4.B_ to B0_4
model4.B_ = B0_4

# Compare model2.B_ and model4.B_
print(np.allclose(model2.B_.todense(), model4.B_.get().todense()))

# Check the shapes of model2.B_ and model4.B_ 
print(model2.B_.shape)
print(model4.B_.get().shape)

True
(1000, 13)
(1000, 13)


In [85]:
RSS = model2.compute_RSS(model2.A_, model2.B_)
RSS_4 = model4.compute_RSS(model4.A_, model4.B_) 

# Compare RSS and RSS_4
print(np.allclose(RSS, RSS_4))

print(RSS)
print(RSS_4)

# Append RSS_4 to model4.RSS_iters 
model4.RSS_iters.append(RSS_4)

False
55.652274644186924
55.65152863338316


In [86]:
model2.step() 
model4.step() 

# Compare model2.A_ and model4.A_
print(np.allclose(model2.A_.todense(), model4.A_.get().todense()))

# Compare model2.B_ and model4.B_
print(np.allclose(model2.B_.todense(), model4.B_.get().todense()))

# print out RSS_iters for model2 and model4 
print(model2.RSS_iters)
print(model4.RSS_iters)

# Compare model2.RSS_iters and model4.RSS_iters
print(np.allclose(model2.RSS_iters, model4.RSS_iters))

False
False
[55.652274644186924, 53.85192071093903]
[55.65152863338316, 53.85609700183744]
False


In [87]:
# Check the convergence threshold for model2 and model4 
print(model2.convergence_threshold)
print(model4.convergence_threshold)

# Set model4.convergence_threshold to model2.convergence_threshold 
model4.convergence_threshold = model2.convergence_threshold 

# Check the convergence threshold for model2 and model4
print(model2.convergence_threshold)
print(model4.convergence_threshold)

0.0005565227464418693
None
0.0005565227464418693
0.0005565227464418693


In [88]:
model2.fit() 

model4.fit()

Randomly initialized A matrix.
Starting iteration 1.
Completed iteration 1.
Starting iteration 10.
Completed iteration 10.


  0%|          | 0/14 [00:00<?, ?it/s]

Converged after 19 iterations.
Building kernel on X_pca
Computing diffusion components from X_pca for waypoint initialization ... 
Determing nearest neighbor graph...
Done.
Sampling waypoints ...
Done.
Selecting 9 cells from waypoint initialization.
Initializing residual matrix using greedy column selection
Initializing f and g...


/home/aparna/miniconda/envs/env_SEACells/lib/python3.8/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '
100%|██████████| 14/14 [00:00<00:00, 20.65it/s]


Selecting 4 cells from greedy initialization.
Randomly initialized A matrix.
Completed iteration 1.
Completed iteration 10.
Completed iteration 20.
Converged after 26 iterations.


In [89]:
# Compare model2.A_ and model4.A_ 
print(np.allclose(model2.A_.todense(), model4.A_.get().todense()))

# Compare model2.B_ and model4.B_
print(np.allclose(model2.B_.todense(), model4.B_.get().todense()))

# Print out model2.RSS_iters and model4.RSS_iters 
print(model2.RSS_iters)
print(model4.RSS_iters)

False
False
[55.652274644186924, 53.85192071093903, 55.65206766839007, 53.847682377926745, 52.19616483662859, 51.847056561438876, 51.668108349302884, 51.52947903398943, 51.43576067281945, 51.37556424367038, 51.3228057292376, 51.27993224482802, 51.2126661456425, 51.12206624670835, 51.020927518668806, 50.94304763730088, 50.90938449827412, 50.88719593141807, 50.87883844328093, 50.86607553907539, 50.86079209469972, 50.86083584001752]
[55.65152863338316, 53.85609700183744, 55.72631266535017, 53.6234116486931, 51.702260923512085, 51.5083485247454, 51.41138607948038, 51.351622034241316, 51.29997025076486, 51.25609989462995, 51.205103986065716, 51.14250514689827, 51.06885832232518, 50.996252175108964, 50.949232599696415, 50.915856479453055, 50.882238168988295, 50.876430910909505, 50.86694230664295, 50.84845766391327, 50.82967196171046, 50.82792083237428, 50.82058097938304, 50.81171690865921, 50.805529089815664, 50.79198838215605, 50.78563493068101, 50.77540941491253, 50.77493962571945]


### Update A

In [90]:
def updateA_compare(B, A_prev):
    n, k = B.shape
    A = A_prev

    Ag = cupyx.scipy.sparse.csc_matrix(A) 
    Bg = cupyx.scipy.sparse.csr_matrix(B)

    K = model2.K
    Kg = model4.K

    t = 0  # current iteration (determine multiplicative update)

    # precompute some gradient terms
    t2 = (K @ B).T
    t1 = t2 @ B

    # precompute the gradient terms for cupy 
    t2g = (Kg.dot(Bg)).T
    t1g = t2g.dot(Bg)

    # Check if t1 and t1g are equal 
    print(np.allclose(t1.todense(), t1g.get().todense())) 

    # Check if t2 and t2g are equal 
    print(np.allclose(t2.todense(), t2g.get().todense()))

    # Create a dataframe that will store the iteration t, is_close_Gg, is_close_amins, is_close_eg, is_close_Ag
    df = pd.DataFrame(columns=["t", "is_close_Gg", "is_close_amins", "is_close_eg", "is_close_Ag", "e", "eg", "G", "Gg"])

    # update rows of A for given number of iterations
    while t < 50:
        # compute gradient (must convert matrix to ndarray)
        G = 2.0 * np.array(t1 @ A - t2)
        # get argmins - shape 1 x n
        amins = np.argmin(G, axis=0)
        amins = np.array(amins).reshape(-1)
        # # loop free implementation
        e = scipy.sparse.csr_matrix((np.ones(len(amins)), (amins, np.arange(n))), shape=A.shape)
        A += 2.0 / (t + 2.0) * (e - A)

        # compute gradient
        Gg = 2.0 * (t1g.dot(Ag) - t2g)

        is_close_Gg = np.allclose(Gg.get().todense(), G.todense())

        aminsG = cp.argmin(Gg.todense(), axis=0)

        is_close_amins = np.allclose(amins, aminsG) 

        # loop free implementaton
        eg = cupyx.scipy.sparse.csr_matrix((cp.ones(len(aminsG)), (aminsG, cp.arange(n))), shape=Ag.shape)
        # eg = cp.zeros((k, n))
        # eg[amins, cp.arange(n)] = 1.0
        # eg = cupyx.scipy.sparse.csr_matrix(eg)

        is_close_eg = np.allclose(eg.get().todense(), e.todense())

        Ag += 2.0/(t+2.0) * (eg - Ag)

        is_close_Ag = np.allclose(Ag.get().todense(), A.todense())

        # Add the iteration t and the is_close_Gg, is_close_amins, is_close_eg, is_close_Ag to the dataframe using df.concat
        df = pd.concat([df, pd.DataFrame([[t, is_close_Gg, is_close_amins, is_close_eg, is_close_Ag, e, eg, G, Gg]], columns=df.columns)], ignore_index=True)

        # # print the iteration t and the is_close_Gg, is_close_amins, is_close_eg, is_close_Ag 
        # print(t, is_close_Gg, is_close_amins, is_close_eg, is_close_Ag)

        t += 1

    return df

In [91]:
A = model2.A0 
B = model2.B0 

df = updateA_compare(B, A)

True
True


/tmp/ipykernel_1184111/2795910654.py:63: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame([[t, is_close_Gg, is_close_amins, is_close_eg, is_close_Ag, e, eg, G, Gg]], columns=df.columns)], ignore_index=True)


In [92]:
df

,t,is_close_Gg,is_close_amins,is_close_eg,is_close_Ag,e,eg,G,Gg
0,0,True,True,True,True,"(0, 4)\t1.0\n (0, 7)\t1.0\n (0, 11)\t1.0\n...","(0, 4)\t1.0\n (0, 7)\t1.0\n (0, 11)\t1.0\n...","(2, 0)\t-2.6106159201089594\n (3, 0)\t0.228...","(0, 4)\t-0.9695073382086843\n (0, 6)\t0.714..."
1,1,True,True,True,True,"(0, 1)\t1.0\n (0, 3)\t1.0\n (0, 5)\t1.0\n ...","(0, 1)\t1.0\n (0, 3)\t1.0\n (0, 5)\t1.0\n ...","(1, 0)\t0.18713144019273928\n (2, 0)\t-0.46...","(0, 4)\t1.5174752756381729\n (0, 6)\t0.0\n ..."
2,2,True,True,True,True,"(0, 0)\t1.0\n (0, 2)\t1.0\n (0, 4)\t1.0\n ...","(0, 0)\t1.0\n (0, 2)\t1.0\n (0, 4)\t1.0\n ...","(1, 0)\t0.0623771467309131\n (2, 0)\t2.1760...","(0, 1)\t3.0560807505124434\n (0, 3)\t3.0560..."
3,3,True,True,True,True,"(0, 4)\t1.0\n (0, 36)\t1.0\n (0, 47)\t1.0\...","(0, 4)\t1.0\n (0, 36)\t1.0\n (0, 47)\t1.0\...","(0, 0)\t2.292060562884332\n (1, 0)\t0.03118...","(0, 0)\t2.2920605628843327\n (0, 1)\t1.5280..."
4,4,True,True,True,True,"(0, 88)\t1.0\n (0, 145)\t1.0\n (0, 158)\t1...","(0, 88)\t1.0\n (0, 145)\t1.0\n (0, 158)\t1...","(0, 0)\t1.375236337730599\n (1, 0)\t0.09356...","(0, 0)\t1.3752363377305996\n (0, 1)\t0.9168..."
5,5,True,True,True,True,"(0, 4)\t1.0\n (0, 27)\t1.0\n (0, 86)\t1.0\...","(0, 4)\t1.0\n (0, 27)\t1.0\n (0, 86)\t1.0\...","(0, 0)\t0.9168242251537327\n (1, 0)\t0.0623...","(0, 0)\t0.9168242251537331\n (0, 1)\t0.6112..."
6,6,True,True,True,True,"(0, 4)\t1.0\n (0, 84)\t1.0\n (0, 134)\t1.0...","(0, 4)\t1.0\n (0, 84)\t1.0\n (0, 134)\t1.0...","(0, 0)\t0.6548744465383806\n (1, 0)\t0.0445...","(0, 0)\t0.6548744465383809\n (0, 1)\t0.4365..."
7,7,True,True,True,True,"(0, 108)\t1.0\n (0, 117)\t1.0\n (0, 145)\t...","(0, 108)\t1.0\n (0, 117)\t1.0\n (0, 145)\t...","(0, 0)\t0.49115583490378545\n (1, 0)\t0.080...","(0, 0)\t0.4911558349037856\n (0, 1)\t0.3274..."
8,8,True,True,True,True,"(0, 4)\t1.0\n (0, 145)\t1.0\n (0, 178)\t1....","(0, 4)\t1.0\n (0, 145)\t1.0\n (0, 178)\t1....","(0, 0)\t0.3820100938140553\n (1, 0)\t0.0623...","(0, 0)\t0.3820100938140555\n (0, 1)\t0.2546..."
9,9,True,True,True,True,"(0, 4)\t1.0\n (0, 85)\t1.0\n (0, 145)\t1.0...","(0, 4)\t1.0\n (0, 85)\t1.0\n (0, 145)\t1.0...","(0, 0)\t0.3056080750512443\n (1, 0)\t0.0499...","(0, 0)\t0.30560807505124443\n (0, 1)\t0.203..."


In [93]:
# Get e and eg from row 1 of df 
e = df.iloc[1]["e"] 
eg = df.iloc[1]["eg"] 

# Compare e and eg 
print(np.allclose(e.todense(), eg.get().todense()))


True


In [94]:
# Compute the sparsity of e and eg 

print("Sparsity of e: ", 1 - (np.count_nonzero(e.todense()) / (e.shape[0] * e.shape[1]))) 
print("Sparsity of eg: ", 1 - (np.count_nonzero(eg.get().todense()) / (eg.shape[0] * eg.shape[1]))) 



Sparsity of e:  0.9230769230769231
Sparsity of eg:  0.9230769230769231


In [95]:
diff = (e.todense() - eg.get().todense()).flatten().tolist()[0]

# Get the frequency of the elements in diff 
freq = {} 
for item in diff:
    if (item in freq):
        freq[item] += 1 
    else:
        freq[item] = 1

print(freq)


{0.0: 13000}


In [96]:
# Get G and Gg from row 1 of df 
G = df.iloc[1]["G"]
Gg = df.iloc[1]["Gg"]

# Compare G and Gg
print(np.allclose(G.todense(), Gg.get().todense()))

True


In [97]:
# compute the sparsity of G and Gg 
print("Sparsity of G: ", 1 - (np.count_nonzero(G.todense()) / (G.shape[0] * G.shape[1])))
print("Sparsity of Gg: ", 1 - (np.count_nonzero(Gg.get().todense()) / (Gg.shape[0] * Gg.shape[1])))

Sparsity of G:  0.8146923076923077
Sparsity of Gg:  0.8146923076923077


In [98]:
diff = (G.todense() - Gg.get().todense()).flatten().tolist()[0]

# Get the frequency of the elements in diff
freq = {}
for item in diff:
    if (item in freq):
        freq[item] += 1
    else:
        freq[item] = 1

print(freq)

{0.0: 12088, -1.7763568394002505e-15: 390, -2.220446049250313e-15: 4, -8.881784197001252e-16: 245, -1.3322676295501878e-15: 18, -1.1102230246251565e-16: 13, -5.551115123125783e-17: 3, 1.1102230246251565e-16: 13, 5.551115123125783e-17: 4, -4.440892098500626e-16: 26, 4.440892098500626e-16: 17, -2.220446049250313e-16: 14, 2.220446049250313e-16: 7, 8.881784197001252e-16: 24, 1.7763568394002505e-15: 48, 3.552713678800501e-15: 13, -3.552713678800501e-15: 43, -2.6645352591003757e-15: 15, 2.6645352591003757e-15: 4, -4.440892098500626e-15: 7, -5.329070518200751e-15: 4}


In [99]:
# aminsG = Gg.argmin(axis=0).ravel() 

# # Calculate aminsG so that we get the lowest possible index for the lowest value in each column of Gg 
# # aminsG = np.argmin(Gg.get().todense(), axis=0).ravel()


# # Compute all the indices of the lowest value in each column of Gg
# minsG = Gg.min(axis=0)
# print(minsG)


aminsG = cp.argmin(Gg.todense(), axis=0)
print(aminsG.shape)

(1000,)


In [100]:
amins = np.argmin(Gg.get(), axis=0)
amins = np.array(amins).reshape(-1)

print(amins)

[ 2  0  8  0  1  0  7  1  0  0  0  1  0  0  1 12  0  1  1  0  0  0  1  0
  0  1  0  2  0  1  1  0  0  1  0  1  3  1  1  0  3  0  0  0  1  0  1  9
  1  1  1  0 12  9  1  1  0  1  1  0  0  0  0  0  7  3  0  0  0  1  0  0
  0  1  3  8  0 11  0  0  1  1  0  1  3  0  0  3  1  0  0  0  0  7  6  9
  0  0  1  0  1  8  0  3  0  0  0  1  3  8  0  1  0  0  7  1  0  2  1  0
  1  0  1  0  1  0  0  9  0  0  7  0 11  0  0  0  0  0  0  1  0  0  0  0
  0  0  9  0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  9  2  1  1  0  1
  0  0  1  0  3  4  0  0  0  0  4  0  4  0  0  0  0  9  3  1  1  0  0  0
  0  0  0  0  0  0  1  1  0  0  9  0  1  1  0  0  3  1  0  1  1  0  0  0
  0  0  0  5  0  0  6  0  1  1  1  0  1  0  0  0  0  0  1  0  0  9  1  0
  1  5  0  8  0  1  0  1  6  0  6  1  0  1  0  3  0  1  0  1  1  0  2  0
  9  0  3  1  8  0  1  0  0  1  0  1  1  1  3  0  1  1  1  0  0  0  0  0
  0  0  0  1  0  1  1  1 10  0  0  0  1  0  6  0  1  0  1  0  0  0  0  1
  0  7  1  0  0  0  0  5  1  0  0  0  3  0 12  1  0

In [101]:
# Compare amins and aminsG 
print(np.allclose(amins, aminsG))

True


In [102]:
Gg.get().todense()[:, 0]

matrix([[ 0.        ],
        [ 0.18713144],
        [-0.46907161],
        [ 0.        ],
        [ 3.62351876],
        [ 4.462217  ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [11.99600696],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]])

In [103]:
# Find the row numbers of every row in aminsG that is not equal to the corresponding row in amins
row_nums = np.where(amins != aminsG.get())[0]
# print(row_nums)

# # Let's make a dataframe that will store the row numbers and the corresponding columns in Gg and G that are not equal. In addition, let's store the amins and aminsG for each row number. 
# df = pd.DataFrame(columns=["row_num", "aminsG", "amins", "Gg"]) 

# for row_num in row_nums:
#     df = pd.concat([df, pd.DataFrame([[row_num,
#                                         aminsG.get()[row_num], 
#                                         amins[row_num], 
#                                         Gg.get().todense()[:, row_num]]], columns=df.columns)], ignore_index=True)

# display(df)

In [104]:
# # Print out the values in the second row of df 
# index = 5
# print("aminsG: ", df.iloc[index]["aminsG"])
# print("amins: ", df.iloc[index]["amins"])
# print("Gg: ", df.iloc[index]["Gg"])

In [105]:
# Where is the behavior of amins and aminsG the same?
# row_nums_same = np.where(amins == aminsG.get())[0] 

# # Make a dataframe that will store the row numbers and the corresponding columns in Gg and G that are equal. In addition, let's store the amins and aminsG for each row number. 
# df_same = pd.DataFrame(columns=["row_num", "aminsG", "amins", "Gg"])

# for row_num in row_nums_same:
#     df_same = pd.concat([df_same, pd.DataFrame([[row_num,
#                                         aminsG.get()[row_num],
#                                         amins[row_num],
#                                         Gg.get().todense()[:, row_num]]], columns=df_same.columns)], ignore_index=True)
    
# display(df_same)

### Fit

In [106]:
# Set model 4 A_ to model 2 A_ 

model4.A_ = cupyx.scipy.sparse.csr_matrix(model2.A_) 

# Set model 4 B_ to model 2 B_ 
model4.B_ = cupyx.scipy.sparse.csr_matrix(model2.B_)

In [107]:
def fit_compare(
        model2, 
        model4, 
        max_iter: int = 50,
        min_iter: int = 10,
        initial_archetypes=None,
        initial_assignments=None,
    ):
        model2.initialize(
            initial_archetypes=initial_archetypes,
            initial_assignments=initial_assignments,
        )

        model4.initialize(
            initial_archetypes=initial_archetypes,
            initial_assignments=initial_assignments,
        ) 

        # Set the model4.A_ to model2.A_ 
        model4.A_ = cupyx.scipy.sparse.csc_matrix(model2.A_) 

        # Set the model4.B_ to model2.B_ 
        model4.B_ = cupyx.scipy.sparse.csr_matrix(model2.B_)

        # Check if model2.A_ and model4.A_ are equal
        print(np.allclose(model2.A_.todense(), model4.A_.get().todense()))

        # Check if model2.B_ and model4.B_ are equal
        print(np.allclose(model2.B_.todense(), model4.B_.get().todense()))

        # Check if model2.K and model4.K are equal 
        print(np.allclose(model2.K.todense(), model4.K.get().todense()))

        # Create a dataframe that will store the iteration n_iter, is_close_A, is_close_B, is_close_K, is_close_RSS
        df = pd.DataFrame(columns=["n_iter", "is_close_A", "is_close_B", "is_close_K", "is_close_RSS", "sparsity_2", "sparsity_4"])

        converged = False
        n_iter = 0

        converged1 = False 
        converged2 = False 

        
        while (not converged and n_iter < max_iter) or n_iter < min_iter:
            n_iter += 1
            if n_iter == 1 or (n_iter) % 10 == 0:
                print(f"Starting iteration {n_iter}.")
            model2.step()
            model4.step() 

            is_close_A = np.allclose(model2.A_.todense(), model4.A_.get().todense()) 
            is_close_B = np.allclose(model2.B_.todense(), model4.B_.get().todense()) 
            is_close_K = np.allclose(model2.K.todense(), model4.K.get().todense()) 
            is_close_RSS = np.allclose(model2.RSS_iters[-1], model4.RSS_iters[-1]) 

            # Compute the sparsity of model2.A_ and model4.A_
            sparsity_2 = model2.A_.count_nonzero()
            sparsity_4 = model4.A_.count_nonzero()

            diff = (model2.A_.todense() - model4.A_.get().todense()).flatten().tolist()[0] 

            # Get the frequency of the elements in diff 
            freq = {}
            for item in diff:
                if (item in freq):
                    freq[item] += 1 
                else:
                    freq[item] = 1

            print(freq)

            # Add the iteration n_iter and the is_close_A, is_close_B, is_close_K, is_close_RSS to the dataframe using df.concat
            df = pd.concat([df, pd.DataFrame([[n_iter, is_close_A, is_close_B, is_close_K, is_close_RSS, sparsity_2, sparsity_4]], columns=df.columns)], ignore_index=True)

            if n_iter == 1 or (n_iter) % 10 == 0:
                print(f"Completed iteration {n_iter}.")

            # Check for convergence
            if (
                cp.abs(model2.RSS_iters[-2] - model2.RSS_iters[-1])
                < model2.convergence_threshold
            ):
                print(f"Converged after {n_iter} iterations.")
                converged1 = True

            if (cp.abs(model4.RSS_iters[-2] - model4.RSS_iters[-1]) < model4.convergence_threshold):
                print(f"Converged after {n_iter} iterations.")
                converged2 = True

            if converged1 and converged2: 
                converged = True

        # Print out the types of model2.A_ and model4.A_ 
        print(type(model2.A_))
        print(type(model4.A_)) 

        # Print out the shapes of model2.A_ and model4.A_ 
        print(model2.A_.shape) 
        print(model4.A_.shape)

        model2.Z_ = model2.B_.T @ model2.K

        # Label cells by SEACells assignment
        labels = model2.get_hard_assignments()
        model2.ad.obs["SEACell"] = labels["SEACell"]

        model4.Z_ = model4.B_.T @ model4.K

        # Label cells by SEACells assignment
        labels = model4.get_hard_assignments()
        model4.ad.obs["SEACell"] = labels["SEACell"]

        if not converged:
            raise RuntimeWarning(
                "Warning: Algorithm has not converged - you may need to increase the maximum number of iterations"
            )
        return df

In [108]:
fit_compare(model2, model4)

Randomly initialized A matrix.
Randomly initialized A matrix.
True
True
True
Starting iteration 1.
{0.0: 12951, 0.011764705882352943: 1, 0.01098039215686275: 1, 0.0015686274509803925: 12, 0.0007843137254902599: 1, -0.02901960784313725: 1, 0.005490196078431372: 1, 0.018823529411764704: 1, -0.0015686274509803925: 12, -0.0007843137254901963: 1, 0.019607843137254905: 1, 0.03058823529411765: 3, 0.02823529411764706: 1, -0.005490196078431372: 1, -0.011764705882352943: 1, 0.010196078431372548: 1, 0.03686274509803921: 1, -0.03058823529411765: 3, -0.01098039215686275: 1, -0.00784313725490196: 1, -0.018823529411764704: 1, -0.010196078431372548: 1, -0.019607843137254905: 1, -0.02823529411764706: 1}
Completed iteration 1.
{0.0: 13000}


/tmp/ipykernel_1184111/2186850751.py:73: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame([[n_iter, is_close_A, is_close_B, is_close_K, is_close_RSS, sparsity_2, sparsity_4]], columns=df.columns)], ignore_index=True)


{0.0: 12976, 0.0007843137254902599: 1, 0.0015686274509803925: 4, 0.008627450980392151: 1, 0.007058823529411763: 1, -0.0007843137254900379: 1, -0.0015686274509803925: 3, 0.007058823529411766: 1, -0.022745098039215688: 1, -0.018039215686274503: 1, -0.0203921568627451: 1, 0.002352941176470589: 1, 0.009411764705882356: 1, 0.02117647058823529: 1, 0.008627450980392158: 1, -0.033725490196078435: 1, 0.022745098039215688: 1, -0.003137254901960785: 1, 0.011764705882352941: 1, -0.002352941176470589: 1}


/tmp/ipykernel_1184111/2186850751.py:73: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame([[n_iter, is_close_A, is_close_B, is_close_K, is_close_RSS, sparsity_2, sparsity_4]], columns=df.columns)], ignore_index=True)


{0.0: 12936, -0.02117647058823529: 1, -0.0031372549019607066: 1, 0.00627450980392158: 1, 0.0039215686274509665: 1, -0.008627450980392165: 1, -0.003137254901960783: 1, 0.0007843137254900379: 1, 0.0023529411764705854: 1, 0.0015686274509803925: 2, -1.1102230246251565e-16: 1, -0.000784313725490593: 1, 0.010196078431372553: 1, -0.0007843137254901766: 1, 0.0203921568627451: 1, -0.0007843137254901905: 1, 0.005490196078431368: 1, 0.004705882352941185: 1, 0.00784313725490195: 1, -0.03372549019607843: 1, 0.0054901960784313735: 1, 0.004705882352941171: 2, 0.008627450980392153: 1, -0.0039215686274509665: 1, -0.013333333333333332: 1, 0.014117647058823526: 1, -0.03294117647058824: 1, -0.018823529411764708: 1, -0.003921568627450981: 1, -0.0023529411764705867: 1, -0.008627450980392158: 1, -0.010196078431372558: 1, -0.00470588235294118: 1, 0.0219607843137255: 1, -0.0054901960784313805: 1, 0.014901960784313736: 1, 0.002352941176470589: 1, -0.024313725490196073: 1, -0.025098039215686273: 1, 0.01882352941

...

{0.0: 3975, 0.00313725490196079: 33, -0.03686274509803911: 2, 0.002352941176470596: 19, 0.03294117647058824: 16, -0.03372549019607843: 21, 0.005490196078431377: 2, 0.003921568627450984: 14, 0.02745098039215685: 2, 0.010980392156862733: 1, 0.014901960784313772: 2, 0.0007843137254901975: 61, -0.006274509803921573: 25, 0.027450980392156862: 31, 0.02901960784313725: 24, 0.0015686274509804018: 23, -0.018039215686274503: 12, -0.04235294117647059: 3, -0.016470588235294112: 11, -0.025882352941176467: 25, -0.04156862745098038: 3, -0.039999999999999994: 4, 0.02980392156862745: 13, -0.0070588235294117615: 3, -0.021176470588235283: 3, -0.03294117647058825: 3, 0.008627450980392179: 5, -0.0023529411764705906: 3, 0.0015686274509803925: 8, -0.007058823529411765: 6, 0.0007843137254902009: 13, 0.0023529411764705924: 29, 0.0023529411764705577: 7, 0.013333333333333343: 11, 0.019607843137254905: 14, 0.006274509803921569: 4, -0.02352941176470589: 22, 0.004705882352941174: 3, 0.029803921568627455: 10, 0.0345

/tmp/ipykernel_1184111/2186850751.py:73: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame([[n_iter, is_close_A, is_close_B, is_close_K, is_close_RSS, sparsity_2, sparsity_4]], columns=df.columns)], ignore_index=True)


{0.0: 4086, 0.031372549019607836: 5, 0.00627450980392158: 14, 0.0023529411764705785: 17, -0.01019607843137249: 3, 0.009411764705882349: 19, 0.023529411764705882: 15, -0.01176470588235294: 3, 0.000784313725490187: 12, -0.023529411764705882: 7, -0.0007843137254901975: 67, 0.021176470588235297: 5, 0.010980392156862757: 2, -0.0023529411764705854: 47, -0.0392156862745098: 17, -0.010196078431372546: 32, 0.033725490196078435: 16, 0.01490196078431371: 1, -0.020392156862745092: 12, -0.017254901960784306: 2, -0.03137254901960784: 42, -0.03372549019607843: 13, -0.0203921568627451: 15, -0.03058823529411763: 1, -0.005490196078431382: 16, -0.007843137254901947: 6, 0.025098039215686277: 6, -0.03686274509803922: 13, 0.01098039215686275: 28, 0.0023529411764705715: 9, 0.005490196078431363: 7, 0.005490196078431375: 34, 0.007058823529411763: 26, -0.021960784313725445: 1, 0.008627450980392148: 6, -0.007058823529411763: 29, 0.001568627450980388: 46, -0.02823529411764708: 1, 0.018039215686274503: 23, -0.0031

/tmp/ipykernel_1184111/2186850751.py:73: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame([[n_iter, is_close_A, is_close_B, is_close_K, is_close_RSS, sparsity_2, sparsity_4]], columns=df.columns)], ignore_index=True)


,n_iter,is_close_A,is_close_B,is_close_K,is_close_RSS,sparsity_2,sparsity_4
0,1,False,True,True,True,11819,11820
1,2,True,True,True,True,8863,8863
2,3,False,True,True,True,7807,7806
3,4,False,True,True,True,7450,7449
4,5,False,True,True,True,7431,7430
5,6,False,False,True,False,7573,7573
6,7,False,False,True,False,7811,7861
7,8,False,False,True,False,8049,8053
8,9,False,False,True,False,8276,8259
9,10,False,False,True,False,8118,8289


In [109]:
A = model4.A_ 

In [110]:
# Compute the sparsity of A 
print("Sparsity of A: ", A.getnnz() / (A.shape[0] * A.shape[1])) 

# Compute the sparsity of A using count_nonzero 
print("Sparsity of A: ", A.count_nonzero() / (A.shape[0] * A.shape[1]))

Sparsity of A:  0.661923076923077
Sparsity of A:  0.661923076923077


In [111]:
A_new = A.eliminate_zeros()

print(A)

  (9, 0)	1.0
  (0, 1)	0.0015686274509803925
  (2, 1)	0.00627450980392157
  (3, 1)	0.03372549019607843
  (6, 1)	0.01254901960784314
  (9, 1)	0.025882352941176467
  (10, 1)	0.9011764705882354
  (12, 1)	0.018823529411764704
  (0, 2)	0.01568627450980392
  (1, 2)	0.03372549019607843
  (2, 2)	0.04705882352941177
  (3, 2)	0.02431372549019608
  (4, 2)	0.003137254901960785
  (5, 2)	0.03215686274509804
  (6, 2)	0.017254901960784316
  (7, 2)	0.09803921568627448
  (8, 2)	0.6227450980392155
  (9, 2)	0.02117647058823529
  (10, 2)	0.035294117647058816
  (11, 2)	0.03607843137254901
  (12, 2)	0.013333333333333336
  (0, 3)	0.02352941176470589
  (1, 3)	0.04784313725490197
  (2, 3)	0.027450980392156862
  (3, 3)	0.04784313725490196
  :	:
  (7, 996)	0.04784313725490198
  (8, 996)	0.06274509803921569
  (9, 996)	0.0203921568627451
  (10, 996)	0.03058823529411765
  (11, 996)	0.11921568627450983
  (12, 996)	0.03686274509803921
  (0, 997)	0.03843137254901961
  (1, 997)	0.06588235294117648
  (2, 997)	0.0674509803

In [112]:
# Calculate the sparsity again 

print("Sparsity of A:", A.getnnz() / (A.shape[0] * A.shape[1]))
print("Sparsity of A:", A.count_nonzero() / (A.shape[0] * A.shape[1]))

Sparsity of A: 0.661923076923077
Sparsity of A: 0.661923076923077
